In [1]:
#!/usr/bin/env python3
import numpy as np
# import PyQt5  # make sure pyqtgraph imports Qt5
# import pyqtgraph
# from PyQt5 import QtCore, QtGui
# from PyQt5.QtWidgets import (
#     QWidget, QHBoxLayout, QVBoxLayout, QTabWidget, QCheckBox
# )
# from PyQt5.QtCore import Qt
# from artiq.applets.simple import TitleApplet
# from scipy.optimize import curve_fit
import pyvisa as visa
from time import sleep
import sys

    

def ramp_mcp_voltage(TODO, V1=2400, V2=2200, V3=200, sleeptime=0.5, print_log=False): 
    rm = visa.ResourceManager()
    instruments = rm.list_resources()
    # instruments
    usb = list(filter(lambda x: 'USB' in x, instruments))
    if len(usb) != 1:
        print('Bad instrument list', instruments)
        sys.exit(-1)
    odp = rm.open_resource(usb[0])
    odp.write("INST:NSEL 1")
    odp.write("VOLT:PROT 10.1")
    odp.write("CURR:PROT 0.001")
    odp.write("OUTP ON")

    odp.write("INST:NSEL 2")
    odp.write("VOLT:PROT 10.1")
    odp.write("CURR:PROT 0.001")
    odp.write("OUTP ON")

    odp.write("INST:NSEL 3")
    odp.write("VOLT:PROT 5")
    odp.write("CURR:PROT 0.001")
    odp.write("OUTP ON")

    Vstep01 = 5*50/5000
    Vstep02 = 5*50/2500 * 1.8
    Vstep03 = 5*50/2500
    V1g0 = 10*V1/5000
    V2g0 = 10*V2/2500
    V3g0 = 10*V3/1250

    ON = "Turn on"
    OFF = "Turn off"

    if TODO == ON:
        odp.write("INST:NSEL 1")
        vinit1 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 2")
        vinit2 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 3")
        vinit3 = float(odp.query("MEAS:VOLT?"))
        while 1:
            odp.write("INST:NSEL 1")
            vcurr1 = float(odp.query("MEAS:VOLT?"))
            sleep(0.2)
            odp.write("INST:NSEL 2")
            vcurr2 = float(odp.query("MEAS:VOLT?"))
            sleep(0.2)
            odp.write("INST:NSEL 3")
            vcurr3 = float(odp.query("MEAS:VOLT?"))
            sleep(0.2)
            if vcurr1 > V1g0+0.1:
                if vcurr2 > V2g0+0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)
                elif vcurr2 < V2g0-0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2+Vstep02))
                        sleep(sleeptime)
                else:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        sleep(sleeptime)
            elif vcurr1 < V1g0-0.1:
                if vcurr2 > V2g0+0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)
                elif vcurr2 < V2g0-0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        sleep(sleeptime)
                else:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1+Vstep01))
                        sleep(sleeptime)       
            else:
                if vcurr2 > V2g0+0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 2")
                        odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)
                elif vcurr2 < V2g0-0.1:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 < (vcurr1*5000/10-50):
                            odp.write("VOLT "+str(vcurr2+Vstep02))
                        sleep(sleeptime)
                else:
                    if vcurr3 > V3g0+0.1:
                        odp.write("INST:NSEL 3")
                        odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    elif vcurr3 < V3g0-0.1:
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250 < vcurr2*2500:
                            odp.write("VOLT "+str(vcurr3+Vstep03))
                        sleep(sleeptime)
                    else:
                        break
            if print_log: 
                print(vcurr1, vcurr2, vcurr3)
        odp.write("INST:NSEL 1")
        odp.write("VOLT "+str(V1g0))
        odp.write("INST:NSEL 2")
        odp.write("VOLT "+str(V2g0))
        odp.write("INST:NSEL 3")
        odp.write("VOLT "+str(V3g0))
        print("Finish")
        if print_log: 
            print(vcurr1, vcurr2, vcurr3)

    elif TODO == OFF:
        odp.write("INST:NSEL 1")
        vinit1 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 2")
        vinit2 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 3")
        vinit3 = float(odp.query("MEAS:VOLT?"))
        while 1:
            odp.write("INST:NSEL 1")
            vcurr1 = float(odp.query("MEAS:VOLT?"))
            sleep(0.1)
            odp.write("INST:NSEL 2")
            vcurr2 = float(odp.query("MEAS:VOLT?"))
            odp.write("INST:NSEL 3")
            vcurr3 = float(odp.query("MEAS:VOLT?"))
            sleep(0.1)
            if vcurr1 > 0.11:
                if vcurr2 > 0.11:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)                    
                else:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 1")
                        odp.write("VOLT "+str(vcurr1-Vstep01))
                        sleep(sleeptime)                    
                    
            else:
                if vcurr2 > 0.11:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        odp.write("INST:NSEL 2")
                        if vcurr2*2500/10 > (vcurr1*5000/10-200):
                            odp.write("VOLT "+str(vcurr2-Vstep02))
                        sleep(sleeptime)                    
                else:
                    if vcurr3 > 0.11:
                        odp.write("INST:NSEL 3")
                        if vcurr3*1250/10 > (vcurr2*2500/10-200):
                            odp.write("VOLT "+str(vcurr3-Vstep03))
                        sleep(sleeptime)
                    else:
                        break
            if print_log:        
                print(vcurr1, vcurr2, vcurr3)
        odp.write("INST:NSEL 1")
        odp.write("VOLT 0.01")
        odp.write("INST:NSEL 2")
        odp.write("VOLT 0.01")                    
        odp.write("INST:NSEL 3")
        odp.write("VOLT 0.01")                    
        
        
    else:
        print("Check your spelling.")

In [3]:
ramp_mcp_voltage("Turn on", V1=2391, V2=2203)

Finish


In [ ]:
ramp_mcp_voltage("Turn off", V1=2391, V2=2203)